<a href="https://colab.research.google.com/github/linbinbin/Langchain_practice/blob/main/colab/Langchain_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-core
!pip install openai
!pip install tiktoken
!pip install langchain-community
!pip install langgraph
!pip install langchain-graph
!pip install langchain-anthropic

In [ ]:
import getpass
import os
from google.colab import userdata
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

#os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["ANTHROPIC_API_KEY"] = userdata.get('CLAUDE_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = "claude_01"
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"


model = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0,
    max_tokens=4096,
    timeout=None,
    max_retries=3,
    # other params...
)

#model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know if there's anything I can assist you with.", response_metadata={'id': 'msg_01MYgZ2Tx9bY8YUeQTBGTeHJ', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 53}}, id='run-f0bcd73c-e365-42ce-9eec-902f07dcf131-0', usage_metadata={'input_tokens': 12, 'output_tokens': 53, 'total_tokens': 65})

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
config = {"configurable": {"session_id": "chat_history_key1"}}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


#with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Jim")], "language": "Japanese"},
    config=config,
)

response.content

'こんにちは、Jimさん。私はあなたの質問に日本語で答えさせていただきます。どのようなことでも質問してください。'

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "English"},
    config=config,
)

response.content

'申し訳ありませんが、私はあなたの名前がJimさんだと理解しています。私の記憶では、最初に自己紹介の際にJimさんと仰っていただきました。私は間違っていますでしょうか?'

In [ ]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm job")], "language": "Chinese"}
)

response.content

'你好,很高兴认识你!我是一个人工智能助手,我会尽我所能回答你的问题。请问你有什么需要帮助的吗?'